In [1]:
from IPython.core.interactiveshell import InteractiveShell
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
%matplotlib inline 
import matplotlib.pyplot as plt
import pandas as pd
import sqlite3
import numpy as np


# Working with SQlite Databases using Python and Pandas


<!-- PELICAN_BEGIN_SUMMARY -->

SQLite is a database engine that makes it simple to store and work with relational data.
Much like the csv format, SQLite stores data in a single file that can be easily shared with others.
Most programming languages and environments have good support for working with SQLite databases.

This example of a life expectancy and birth rate by country file that downloaded from federal government website.
The file is in Excel format.

<!-- PELICAN_END_SUMMARY -->


## Sample is the Life Expectency and Population by Country
- Query Databases Row in Python
- Mapping life expedency over 100 years period 
- Reading results into a pandas DataFrame
- Modifying database rows
- Inserting rows with Python
- Passing parameters into a query
- Updating rows
- Deleting rows
- Creating tables
- Creating tables with pandas
- Altering tables with Pandas
- Visualize chart
- Export out the file to Excel



In [2]:
#Name of Excel xlsx file. SQLite database will have the same name and extension .db
#Connect SQlite file - make changes in Excel, will automatically update in SQLite after reconnect
filename="data/Life" 
con=sqlite3.connect(filename+".db")  

wb=pd.read_excel(filename+'.xlsx',sheet_name=None)
for sheet in wb:
    wb[sheet].to_sql(sheet,con, index=False, if_exists='replace')
  
    
con.commit()
#con.close()


C:\apps\python36\python-3.6.3.amd64\lib\site-packages\pandas\core\generic.py:1534: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  chunksize=chunksize, dtype=dtype)


In [3]:
# view the table from SQLite and set the index key to "CountryName"

df_life_expectancy = pd.read_sql('select * from LifeExpectancy', con, index_col="CountryName")
df_life_expectancy.head()



,CountryCode,Year_1960,Year_1961,Year_1962,Year_1963,Year_1964,Year_1965,Year_1966,Year_1967,Year_1968,...,Year_2006,Year_2007,Year_2008,Year_2009,Year_2010,Year_2011,Year_2012,Year_2013,Year_2014,Year_2015
CountryName,,,,,,,,,,,,,,,,,,,,,
Aruba,ABW,65.569366,65.988024,66.365537,66.713976,67.044293,67.369756,67.699000,68.034683,68.377146,...,74.338829,74.482756,74.627683,74.771146,74.910634,75.047659,75.183195,75.319732,75.456780,75.594341
Afghanistan,AFG,32.337561,32.786976,33.228927,33.666902,34.101902,34.534902,34.966927,35.398927,35.832415,...,59.125951,59.708854,60.256732,60.764610,61.235463,61.674244,62.092976,62.501610,62.902683,63.298195
Angola,AGO,33.226024,33.547756,33.888000,34.244732,34.617463,35.002220,35.396512,35.797854,36.202268,...,53.927122,55.047512,56.139293,57.181000,58.141122,58.991244,59.718439,60.321756,60.806732,61.189341
Albania,ALB,62.254366,63.273463,64.162854,64.887098,65.438195,65.827390,66.089317,66.287220,66.474951,...,75.561244,75.794927,76.072659,76.394878,76.747902,77.108463,77.448268,77.747098,77.998390,78.203146
Andorra,AND,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# we can read the first table from Excel file too, the difference is we can set primary key when using SQLite
df = pd.read_excel('data/Life.xlsx')
df.head()

,CountryName,CountryCode,Year_1960,Year_1961,Year_1962,Year_1963,Year_1964,Year_1965,Year_1966,Year_1967,...,Year_2006,Year_2007,Year_2008,Year_2009,Year_2010,Year_2011,Year_2012,Year_2013,Year_2014,Year_2015
0,Aruba,ABW,65.569366,65.988024,66.365537,66.713976,67.044293,67.369756,67.699000,68.034683,...,74.338829,74.482756,74.627683,74.771146,74.910634,75.047659,75.183195,75.319732,75.456780,75.594341
1,Afghanistan,AFG,32.337561,32.786976,33.228927,33.666902,34.101902,34.534902,34.966927,35.398927,...,59.125951,59.708854,60.256732,60.764610,61.235463,61.674244,62.092976,62.501610,62.902683,63.298195
2,Angola,AGO,33.226024,33.547756,33.888000,34.244732,34.617463,35.002220,35.396512,35.797854,...,53.927122,55.047512,56.139293,57.181000,58.141122,58.991244,59.718439,60.321756,60.806732,61.189341
3,Albania,ALB,62.254366,63.273463,64.162854,64.887098,65.438195,65.827390,66.089317,66.287220,...,75.561244,75.794927,76.072659,76.394878,76.747902,77.108463,77.448268,77.747098,77.998390,78.203146
4,Andorra,AND,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# now lets take a looks the second table and set the primary key to "CountryName"
df_population = pd.read_sql('select * from Population', con, index_col="CountryName")
df_population.head()



,Population,Area_sq_mi,Pop_Density_per_sq_mi,Coastline_coast_area_ratio,Net_migration,Infant_mortality_per_1000 births,GDP_ per_capita,Literacy,Birthrate,Deathrate,Agriculture,Industry,Service
CountryName,,,,,,,,,,,,,
Aruba,31056997,647500,47.96,0.00,23.06,163.07,700.0,36.0,46.60,20.34,0.380,0.240,0.380
Afghanistan,3581655,28748,124.59,1.26,-4.93,21.52,4500.0,86.5,15.11,5.22,0.232,0.188,0.579
Angola,32930091,2381740,13.83,0.04,-0.39,31.00,6000.0,70.0,17.14,4.61,0.101,0.600,0.298
Albania,57794,199,290.42,58.29,-20.71,9.27,8000.0,97.0,22.46,3.27,NaN,NaN,NaN
Andorra,71201,468,152.14,0.00,6.60,4.05,19000.0,100.0,8.71,6.25,NaN,NaN,NaN


In [9]:
# then we can join table to do a query and save the result to a third table or save as Excel format
sql_statement='''
select l.CountryName, l.year_1970, p.Birthrate
from LifeExpectancy as l
join Population as p on p.CountryName=l.CountryName;
'''

df_results = pd.read_sql(sql_statement, con, index_col="CountryName")
df_results.head()



,Year_1960,Birthrate
CountryName,,
Aruba,65.569366,46.60
Afghanistan,32.337561,15.11
Angola,33.226024,17.14
Albania,62.254366,22.46
Andorra,NaN,8.71


In [10]:
# we can create a new table for our query result and add it to the existing Life.db
df_results.to_sql("pop_life_expectancy",con, index="CountryName", if_exists='replace')
con.commit()
#con.close()


## Here is table view from DB Browser

<img src="/images/LifeNewTable.png" alt="[img: DB Browser after creating new query result to a table]" title="DbBrowser Query" />